# Open AI Chat

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys

In [3]:
sys.path.insert(0, '/home/eugene/src/langchain/libs/langchain/')

In [4]:
from typing import List, Optional, Any
from langchain.tools import tool, Tool, format_tool_to_openai_function
from langchain.schema.messages import AIMessage, HumanMessage
from langchain.chat_models.openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

from langchain.automaton.typedefs import MessageLog
from langchain.automaton.tool_utils import generate_tool_info
from langchain.automaton.openai_agent import OpenAIAgent
from langchain.chat_models.openai import ChatOpenAI

In [5]:
def get_tools() -> List[Tool]:
    @tool
    def name() -> str:
        """Use to look up the user's name"""
        return "Eugene"

    @tool
    def get_weather(city: str) -> str:
        """Get weather in a specific city."""
        return "42F and sunny"

    @tool
    def add(x: int, y: int) -> int:
        """Use to add two numbers. For example; { "x": 2, "y": 10} """
        return x + y

    @tool
    def add_3(x: int, y: int, z: int) -> int:
        """Add 3 numbers together."""
        return x + y + z

    @tool
    def divide(x: float, y: float) -> float:
        """Divide x by y"""
        return x / y

    return list(locals().values())

tools = get_tools()

In [6]:
openai_funcs = [format_tool_to_openai_function(tool_) for tool_ in tools]
llm = ChatOpenAI(temperature=0).bind(functions=openai_funcs)

In [7]:
agent = OpenAIAgent(llm, tools)

In [8]:
a = agent.invoke([HumanMessage(content="Sum of first 10 numbers starting from 1? use one tool at a time")])

In [18]:
for msg in a:
    print(msg)

ai: {'function_call': {'name': 'add', 'arguments': '{\n  "x": 1,\n  "y": 2\n}'}}
FunctionCall(name=add, named_arguments={'x': 1, 'y': 2})
FunctionResult(name=add, result=3, error=None)
ai: {'function_call': {'name': 'add', 'arguments': '{\n  "x": 3,\n  "y": 3\n}'}}
FunctionCall(name=add, named_arguments={'x': 3, 'y': 3})
FunctionResult(name=add, result=6, error=None)
ai: {'function_call': {'name': 'add', 'arguments': '{\n  "x": 6,\n  "y": 4\n}'}}
FunctionCall(name=add, named_arguments={'x': 6, 'y': 4})
FunctionResult(name=add, result=10, error=None)
ai: {'function_call': {'name': 'add', 'arguments': '{\n  "x": 10,\n  "y": 5\n}'}}
FunctionCall(name=add, named_arguments={'x': 10, 'y': 5})
FunctionResult(name=add, result=15, error=None)
ai: {'function_call': {'name': 'add', 'arguments': '{\n  "x": 15,\n  "y": 6\n}'}}
FunctionCall(name=add, named_arguments={'x': 15, 'y': 6})
FunctionResult(name=add, result=21, error=None)
ai: {'function_call': {'name': 'add', 'arguments': '{\n  "x": 21,\n 